In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
df = spark.read.format("delta").load("/Volumes/workspace/bronze/bronzevolume/airports/data")

# df = df.withColumn("flight_date", to_date(col("flight_date")))\
#     .drop("_rescued_data")\
#     .withColumn("modifiedDate", current_timestamp())

display(df)

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-6394372965142097>, line 7
      1 df = spark.read.format("delta").load("/Volumes/workspace/bronze/bronzevolume/passengers/data")
      3 # df = df.withColumn("flight_date", to_date(col("flight_date")))\
      4 #     .drop("_rescued_data")\
      5 #     .withColumn("modifiedDate", current_timestamp())
----> 7 display(df)

File /databricks/python_shell/lib/dbruntime/display.py:142, in Display.display(self, input, *args, **kwargs)
    140 # This version is for Serverless + Spark Connect dogfooding.
    141 elif self.spark_connect_enabled and isinstance(input, ConnectDataFrame):
--> 142     self.display_connect_table(input, **kwargs)
    143 elif isinstance(input, ConnectDataFrame):
    144     if input.isStreaming:

File /databricks/python_shell/lib/dbruntime/display.py:103, in Display.display_connect_table(self, df, **kwar

In [0]:
%sql
SELECT * from workspace.silver.silver_passengers where passenger_id = 'P0049'

passenger_id,name,gender,nationality,modifiedDate
P0049,Douglas Montoya,Female,Pakistan,2025-07-18T19:41:36.272Z


In [0]:
import dlt
from pyspark.sql.functions import *
from pyspark.sql.types import *


The Delta Live Tables (DLT) module is not supported on this cluster.
 You should either create a new pipeline or use an existing pipeline to run DLT code.

The Delta Live Tables (DLT) module is not supported on this cluster.
 You should either create a new pipeline or use an existing pipeline to run DLT code.

---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
File <command-6394372965142086>, line 1
----> 1 import dlt
      2 from pyspark.sql.functions import *
      3 from pyspark.sql.types import *

File /databricks/python_shell/lib/dbruntime/autoreload/discoverability/hook.py:71, in AutoreloadDiscoverabilityHook._patched_import(self, name, *args, **kwargs)
     65 if not self._should_hint and (
     66     (module := sys.modules.get(absolute_name)) is not None and
     67     (fname := get_allowed_file_name_or_none(module)) is not None and
     68     (mtime := os.stat(fname).st_mtime) > self.last_mtime_by_modname.get(
     69         absolute_name, float("inf")) and not self._should_hint):
     70     self._should_hint = True
---> 71 module = self._original_builtins_import(name, *args, **kwargs)
     72 if (fname := fname or get_allowed_file_name_or_none(module)) is not None:
     73     

In [0]:
@dlt.table(
    name="stage_bookings", 
    comment="This table contains all bookings",
)
def stage_bookings():
    df = spark.readStream.format('delta')\
        .load('/Volumes/workspace/bronze/bronzevolume/bookings/data')
    return df

In [0]:
@dlt.view(
    name="trans_bookings"
)
def trans_bookings():
    df = spark.readStream.table("stage_bookings")
    df= df\
        .withColumn("amount", col("amount").cast(DoubleType()))\
        .withColumn("modifiedDate", current_timestamp())\
        .withColumn("booking_date", to_date(col("booking_date")))\
        .drop("_rescued_data")
    return df

In [0]:
rules = {
    "rule1": "booking_id IS NOT NULL",
    "rule2": "passenger_id IS NOT NULL",
}

In [0]:
@dlt.table(
    name="silver_bookings"
)
@dlt.expect_all_or_drop(rules)
def silver_bookings():
    df = dlt.read("trans_bookings")
    return df